In [1]:
from fastapi import FastAPI, Query
import pandas as pd
import os
from fastapi.responses import JSONResponse, FileResponse


In [4]:
def create_charts_jsons(startdate, enddate):
    # Load the dataset
    df = pd.read_csv('data_cleaning_and_queries' + os.sep + 'artVis_data_cleaned.csv')
    
    # Filter for the specified years
    filtered_df = df.loc[(df['e_startdate'] >= startdate) & (df['e_startdate'] <= enddate)]

    grouped_by_artist = filtered_df.groupby(['a_id']).agg({'a_firstname': 'first',
                                                           'a_lastname': 'first',
                                                           'e_paintings': 'sum',
                                                           'e_id': 'count',
                                                           'e_venue': lambda x: x.nunique()})

    grouped_by_venue = filtered_df.groupby(['e_venue']).agg({'e_country': 'first',
                                                             'e_city': 'first',
                                                             'a_id': lambda x: x.nunique(),
                                                             'e_id': lambda x: x.nunique(),
                                                             'e_paintings': 'sum'})

    json_files = [grouped_by_artist.to_json(orient='records'), grouped_by_venue.to_json(orient='records')]

    # Save the dataframes as an HTML file
    charts_urls = ["grouped_by_artist.html", "grouped_by_venue.html"]
    for i in [0, 1]:
        json_files[i].to_html(charts_urls[i])
    
    return charts_urls

In [5]:
create_charts_jsons(1908, 1912)

AttributeError: 'str' object has no attribute 'to_html'

 Volume in drive C is Windows-SSD
 Volume Serial Number is E4F7-167F

 Directory of C:\Users\suzan\PycharmProjects\TU_projects\Information_Visualization\new

2025-01-17  11:38    <DIR>          .
2025-01-16  22:35    <DIR>          ..
2025-01-16  20:35             2,840 charts.js
2025-01-16  20:24             2,426 dashboard.js
2025-01-16  10:40           402,387 Dashboard_Sketch.png
2025-01-15  22:41    <DIR>          data
2025-01-16  17:31             4,119 index.html
2025-01-16  16:37             2,636 load_data.js
2025-01-16  18:58               996 map.js
2025-01-16  17:26             2,163 map.py
2025-01-14  23:02    <DIR>          node_modules
2025-01-14  23:02               843 package-lock.json
2025-01-16  03:20               363 package.json
2025-01-14  12:50               410 README.txt
2025-01-16  11:08                64 requirements.txt
2025-01-16  20:24             1,292 slider.js
2025-01-15  04:28             1,560 style.css
2025-01-16  19:38    <DIR>          __pycache_

In [ ]:

app = FastAPI()

@app.get("/get-charts-data")
async def get_charts_data(
    startYear: int = Query(..., description="The start year for filtering"),
    endYear: int = Query(..., description="The end year for filtering"),
):
    """
    Endpoint to fetch charts data filtered by a year range.
    """
    # Call the `create_chart` function
    charts_file = create_charts(startYear, endYear)
    #return FileResponse(charts_file, media_type="text/html", filename=os.path.basename(charts_file))

    # Return the URL to the charts (no need for the file response)
    return JSONResponse(content={"charts_url": charts_url})
    
